In [3]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
np.random.seed(42)

In [5]:
url = 'https://raw.githubusercontent.com/neurodatascience/course-2020-assessments/master/practical/brainsize.csv'
df = pd.read_csv(url, sep=';', na_values='.', index_col=0)

print(df.head(5))
print(len(df.index))

   Gender  FSIQ  VIQ  PIQ  Weight  Height  MRI_Count
1  Female   133  132  124   118.0    64.5     816932
2    Male   140  150  124     NaN    72.5    1001121
3    Male   139  123  150   143.0    73.3    1038437
4    Male   133  129  128   172.0    68.8     965353
5  Female   137  132  134   147.0    65.0     951545
40


In [6]:
partY = np.random.normal(loc=0.0, scale=1.0, size=len(df.index))
df.insert(len(df.columns), 'partY', partY)
print(df.head(5))

   Gender  FSIQ  VIQ  PIQ  Weight  Height  MRI_Count     partY
1  Female   133  132  124   118.0    64.5     816932  0.496714
2    Male   140  150  124     NaN    72.5    1001121 -0.138264
3    Male   139  123  150   143.0    73.3    1038437  0.647689
4    Male   133  129  128   172.0    68.8     965353  1.523030
5  Female   137  132  134   147.0    65.0     951545 -0.234153


In [7]:
not_partY = list(df.columns[0:len(df.columns)-1])
print(not_partY)

['Gender', 'FSIQ', 'VIQ', 'PIQ', 'Weight', 'Height', 'MRI_Count']


In [0]:
import itertools
def generate_formula_ends(df):
  variable_combos = []
  formula_ends = []
  not_partY = list(df.columns[0:len(df.columns)-1])
  count = len(not_partY) - 1
  while count > 0:
    for combination in itertools.combinations(not_partY, count):
        variable_combos.append(combination)
    count -= 1
  for variable_combo in variable_combos:
    for index in range(len(variable_combo)):
      if index == 0:
        formula_string = str(variable_combo[index])
      else:
        formula_string = formula_string + ' + ' + variable_combo[index]
    formula_ends.append(formula_string)
  return formula_ends

In [0]:
def gen_dict_p(df):
  dict_formulas = {}
  formula_ends = generate_formula_ends(df)
  for formula_end in formula_ends:
    formula = 'partY ~ ' + formula_end
    model = ols(formula, data=df)
    results = model.fit()
    dict_formulas[formula_end] = results.pvalues
  return dict_formulas

In [0]:
def find_lowest(dict_formulas):
  lowest_p_value = 42
  lowest_formula_end = ""
  for formula_ends, p_values in dict_formulas.items():
      min_p_value = min(p_values)
      if min_p_value < lowest_p_value:
          lowest_p_value = min_p_value
          lowest_formula_end = formula_ends
  return [lowest_p_value, lowest_formula_end]

In [0]:
def check_p_for_first(df):
  dict_formulas = gen_dict_p(df)
  lowest_formula = find_lowest(dict_formulas)
  # see if excluding first participant leads to lower p value
  recruited_minus_first = df.tail(len(df) - 1)
  dict_formulas_minus_first = gen_dict_p(recruited_minus_first)
  lowest_formula_minus_first = find_lowest(dict_formulas_minus_first)
  return ['With first', lowest_formula, 'Without first', lowest_formula_minus_first]

In [19]:
compare_excluding = check_p_for_first(df)
print(compare_excluding)

['With first', [0.0047084705657578, 'PIQ + Weight'], 'Without first', [0.0031635498420755645, 'PIQ + Weight']]


In [21]:
model = ols('partY ~ PIQ + Weight', data=df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  partY   R-squared:                       0.191
Model:                            OLS   Adj. R-squared:                  0.145
Method:                 Least Squares   F-statistic:                     4.134
Date:                Sun, 31 May 2020   Prob (F-statistic):             0.0244
Time:                        19:01:16   Log-Likelihood:                -46.908
No. Observations:                  38   AIC:                             99.82
Df Residuals:                      35   BIC:                             104.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.5064      1.161     -3.019      0.005      -5.864      -1.149
PIQ            0.0138      0.006      2.186      0.036       0.001       0.027
Weight         0.0113      0.006      1.863      0.071      -0.001       0.024
==============================================================================
Omnibus:                        0.663   Durbin-Watson:                   2.013
Prob(Omnibus):                  0.718   Jarque-Bera (JB):                0.203
Skew:                          -0.166   Prob(JB):                        0.903
Kurtosis:                       3.136   Cond. No.                     1.56e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [0]:
new_df = df.drop(1)

In [23]:
model = ols('partY ~ PIQ + Weight', data=new_df).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  partY   R-squared:                       0.208
Model:                            OLS   Adj. R-squared:                  0.161
Method:                 Least Squares   F-statistic:                     4.458
Date:                Sun, 31 May 2020   Prob (F-statistic):             0.0191
Time:                        19:01:18   Log-Likelihood:                -45.440
No. Observations:                  37   AIC:                             96.88
Df Residuals:                      34   BIC:                             101.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -3.7133      1.169     -3.177      0.003      -6.089      -1.338
PIQ            0.0131      0.006      2.074      0.046       0.000       0.026
Weight         0.0130      0.006      2.095      0.044       0.000       0.026
==============================================================================
Omnibus:                        0.655   Durbin-Watson:                   2.085
Prob(Omnibus):                  0.721   Jarque-Bera (JB):                0.139
Skew:                          -0.115   Prob(JB):                        0.933
Kurtosis:                       3.193   Cond. No.                     1.56e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.56e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""